In [ ]:
!pip install transformers torch scikit-learn pandas

# Importing the Necessary libraries

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW  # <-- Use PyTorch's version instead

# Your existing code will work exactly the same way:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
optimizer = AdamW(model.parameters(), lr=5e-5)  # Same syntax!
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import numpy as np


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Loading and Preprocessing the Dataset

In [ ]:
# Load your cleaned dataset
df = pd.read_csv("/content/Industrial_safety_and_health_database_with_accidents_description.csv")  # Replace with your actual filename
df = df[['Description', 'Accident Level']].dropna()

# Encode target labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Accident Level'])

X_train, X_test, y_train, y_test = train_test_split(df['Description'], df['label'], test_size=0.2, random_state=42)


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=128)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
class AccidentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = AccidentDataset(train_encodings, list(y_train))
test_dataset = AccidentDataset(test_encodings, list(y_test))


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_encoder.classes_))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
epochs = 32
model.train()

for epoch in range(epochs):
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} Average Loss: {avg_loss:.4f}")


Training Epoch 1: 100%|██████████| 43/43 [00:08<00:00,  5.03it/s]


Epoch 1 Average Loss: 0.9911


Training Epoch 2: 100%|██████████| 43/43 [00:07<00:00,  5.66it/s]


Epoch 2 Average Loss: 0.9496


Training Epoch 3: 100%|██████████| 43/43 [00:07<00:00,  5.60it/s]


...

Epoch 30 Average Loss: 0.0128


Training Epoch 31: 100%|██████████| 43/43 [00:08<00:00,  5.27it/s]


Epoch 31 Average Loss: 0.0132


Training Epoch 32: 100%|██████████| 43/43 [00:08<00:00,  5.28it/s]

Epoch 32 Average Loss: 0.0127


In [ ]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("Classification Report:\n", classification_report(all_labels, all_preds, target_names=label_encoder.classes_))
print("Confusion Matrix:\n", confusion_matrix(all_labels, all_preds))


Evaluating: 100%|██████████| 11/11 [00:00<00:00, 16.89it/s]

Classification Report:
               precision    recall  f1-score   support

           I       0.81      0.87      0.84        68
          II       0.00      0.00      0.00         6
         III       0.00      0.00      0.00         5
          IV       0.00      0.00      0.00         4
           V       0.00      0.00      0.00         2

    accuracy                           0.69        85
   macro avg       0.16      0.17      0.17        85
weighted avg       0.65      0.69      0.67        85

Confusion Matrix:
 [[59  4  3  2  0]
 [ 6  0  0  0  0]
 [ 3  1  0  1  0]
 [ 4  0  0  0  0]
 [ 1  1  0  0  0]]



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
model.save_pretrained("bert_accident_model_manual")
tokenizer.save_pretrained("bert_accident_model_manual")

import joblib
joblib.dump(label_encoder, "label_encoder_manual.pkl")


['label_encoder_manual.pkl']

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import joblib

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model, tokenizer, label encoder
model = BertForSequenceClassification.from_pretrained("bert_accident_model_manual")
tokenizer = BertTokenizer.from_pretrained("bert_accident_model_manual")
label_encoder = joblib.load("label_encoder_manual.pkl")

model.to(device)
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
def predict_severity(description):
    inputs = tokenizer(description, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        pred_class = torch.argmax(probs, dim=1).item()
        label = label_encoder.inverse_transform([pred_class])[0]

    return label, probs.cpu().numpy()[0]


In [ ]:
description = "The employee had headache"
severity, confidence = predict_severity(description)
print(f"Predicted Severity: {severity}")
print(f"Confidence: {confidence}")


Predicted Severity: I
Confidence: [9.9963868e-01 1.0083882e-04 7.6366523e-05 1.2578002e-04 5.8215555e-05]


In [ ]:
!pip install gradio


In [ ]:
import gradio as gr

def chatbot_interface(description):
    severity, conf = predict_severity(description)
    conf_percent = [f"{label}: {round(score*100, 2)}%" for label, score in zip(label_encoder.classes_, conf)]
    return f"Predicted Severity: {severity}\n\nConfidence:\n" + "\n".join(conf_percent)

gr.Interface(fn=chatbot_interface,
             inputs=gr.Textbox(label="Describe the incident"),
             outputs="text",
             title="Industrial Safety Severity Predictor").launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ae4a8fa6fec6a204cf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Correcting the Imbalance in data set

In [ ]:
import torch
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Accident Level'])

# Step 1: Convert descriptions to BERT embeddings
inputs = tokenizer(
    df['Description'].tolist(),
    padding=True,
    truncation=True,
    return_tensors="pt"
)

with torch.no_grad():
    outputs = model.bert(**inputs)   # use the encoder part of BertForSequenceClassification
    embeddings = outputs.last_hidden_state[:,0,:].numpy()  # CLS token embeddings

# Step 2: Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(embeddings, y)

# Step 3: Rebuild resampled DataFrame
resampled_df = pd.DataFrame({
    'Accident Level': label_encoder.inverse_transform(y_resampled)
})
# Add text back (sampled to match resampled size)
resampled_df['Description'] = df.sample(len(resampled_df), replace=True)['Description'].values

print("Original class distribution:")
print(pd.Series(y).value_counts())
print("\nResampled class distribution:")
print(pd.Series(y_resampled).value_counts())


RuntimeError: Expected all tensors to be on the same device, but got index is on cpu, different from other tensors on cuda:0 (when checking argument in method wrapper_CUDA__index_select)

In [ ]:
resampled_df.to_csv("cleaned_data.csv", index=False)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

loss_fn = nn.CrossEntropyLoss(weight=class_weights)


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    resampled_df['Description'].tolist(),
    label_encoder.transform(resampled_df['Accident Level']),
    stratify=resampled_df['Accident Level'],
    test_size=0.2,
    random_state=42
)


In [ ]:
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Load your cleaned dataset
df = pd.read_csv("/content/cleaned_data.csv")

# Encode labels
label_encoder = LabelEncoder()
df['Accident Level Encoded'] = label_encoder.fit_transform(df['Accident Level'])

# Reshape description for oversampling
X = df['Description'].values.reshape(-1, 1)
y = df['Accident Level Encoded'].values

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

resampled_df = pd.DataFrame({
    'Description': X_resampled.flatten(),
    'Accident Level Encoded': y_resampled
})
resampled_df['Accident Level'] = label_encoder.inverse_transform(y_resampled)

# Save for reference
resampled_df.to_csv("resampled_accidents.csv", index=False)

resampled_df['Accident Level'].value_counts()


,count
Accident Level,
I,316
IV,316
III,316
II,316
V,316


In [ ]:
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

X_train, X_val, y_train, y_val = train_test_split(
    resampled_df['Description'].tolist(),
    resampled_df['Accident Level Encoded'].tolist(),
    stratify=resampled_df['Accident Level Encoded'],
    test_size=0.2,
    random_state=42
)

train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=128)


In [ ]:
import torch

class AccidentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} | {
            "labels": torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

train_dataset = AccidentDataset(train_encodings, y_train)
val_dataset = AccidentDataset(val_encodings, y_val)


In [ ]:
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader
from torch.optim import AdamW  # <-- Use PyTorch's version instead
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))
model.to(device)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

optimizer = AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(3):
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 2: 100%|██████████| 79/79 [00:28<00:00,  2.79it/s, loss=0.03]


In [ ]:
model.save_pretrained("bert_accident_model_balanced")
tokenizer.save_pretrained("bert_accident_model_balanced")

import joblib
joblib.dump(label_encoder, "label_encoder_balanced.pkl")


['label_encoder_balanced.pkl']

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert_accident_model_balanced")
tokenizer = BertTokenizer.from_pretrained("bert_accident_model_balanced")
label_encoder = joblib.load("label_encoder_balanced.pkl")

model.to(device)
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
def predict_severity(description):
    inputs = tokenizer(description, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        pred_class = torch.argmax(probs, dim=1).item()
        label = label_encoder.inverse_transform([pred_class])[0]

    return label, probs.cpu().numpy()[0]


In [ ]:
!pip install gradio


In [ ]:
import gradio as gr

def chatbot_interface(description):
    severity, conf = predict_severity(description)
    conf_percent = [f"{label}: {round(score*100, 2)}%" for label, score in zip(label_encoder.classes_, conf)]
    return f"Predicted Severity: {severity}\n\nConfidence:\n" + "\n".join(conf_percent)

gr.Interface(
    fn=chatbot_interface,
    inputs=gr.Textbox(label="Describe the accident"),
    outputs="text",
    title="Industrial Safety Chatbot"
).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cafe7672d8ff4beaca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import torch

def generate_response(description, model, tokenizer, label_encoder):
    # Encode and predict
    inputs = tokenizer(description, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1).detach().numpy()[0]

    # Get top prediction and top 2 alternatives
    top_indices = probs.argsort()[-3:][::-1]
    top_labels = label_encoder.inverse_transform(top_indices)
    top_probs = probs[top_indices]

    # Natural language response
    response = f"Predicted accident severity is **{top_labels[0]}** ({top_probs[0]*100:.1f}%)."

    if len(top_labels) > 1:
        response += f" Other possibilities include {top_labels[1]} ({top_probs[1]*100:.1f}%) and {top_labels[2]} ({top_probs[2]*100:.1f}%)."

    # Basic explanation from description keywords
    desc_lower = description.lower()
    if "fall" in desc_lower:
        response += " The mention of a fall suggests potential impact-related injuries."
    elif "fire" in desc_lower or "burn" in desc_lower:
        response += " References to fire/burns indicate heat-related severity."
    elif "machine" in desc_lower or "pulley" in desc_lower:
        response += " The machinery context suggests mechanical hazard risks."

    return response

# Example usage
example_desc = "Pulley fell and injured worker's foot"
print(generate_response(example_desc, model, tokenizer, label_encoder))


In [ ]:
# Generate responses for the full dataset and add as a new column
resampled_df['Predicted Response'] = resampled_df['Description'].apply(
    lambda x: generate_response(x, model, tokenizer, label_encoder)
)

# Show sample outputs
resampled_df[['Description', 'Accident Level', 'Predicted Response']].head()
